In [1]:
from extract_reductions_adult import *


Fair on all test
Accuracy is 0.830625
Balanced accuracy is 0.740520
Gap RMS is 0.031331694965181595
Mean absolute gap is 0.025579379162447194
Max gap is 0.043672761130861715
Average odds difference is 0.018093
Equal opportunity difference is 0.043673
Statistical parity difference is -0.097299


In [8]:
weights = [a.tolist() for a in coefs]
weights

[[0.37048889356855286,
  0.7308070431791421,
  2.333673211318028,
  0.2427125666705313,
  0.3702697747197795,
  0.5789813784857318,
  -0.19385643445079598,
  -0.10337929364832746,
  -0.021800306966834073,
  -0.7080693887502542,
  -0.2792874862478641,
  -0.8237897251416941,
  -0.9034522443478414,
  1.2216912673829858,
  1.1998498475960635,
  -0.29770118816385066,
  -1.2421449643041873,
  -0.6344081572201314,
  -0.8950358176774175,
  -0.04969160210912986,
  0.08384247584615481,
  0.09253037791968462,
  0.7520823705325481,
  -1.0170637128007387,
  -0.6220208454827904,
  -0.2848880928304454,
  -0.6185748425886097,
  -1.784071310636403,
  0.5591501177437977,
  0.5959449536306793,
  0.23905086579725132,
  0.5356767652325941,
  -0.03316877697916665,
  -0.23077918285228707,
  -0.04920769553343867,
  -0.7667875065498329,
  -1.1296841230128574,
  -0.06977866217923372,
  0.6950359134014691],
 [0.3656486777095961,
  0.7260134153536855,
  2.3229935701289954,
  0.23831354198213883,
  0.3699816176003

In [9]:
data = {'ens_weights': ens_weights, 'intercepts': intercepts, 'coefs': weights}

In [11]:
import json
with open('data.txt', 'w') as f:
    json.dump(data, f)
f.close()

In [12]:
with open('data.txt', 'r') as f:
    data1 = json.load(f)
f.close()

In [15]:
with open('text.txt', 'r') as f:
    

In [1]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import model
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()


seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))





## Running linear regression to get sensetive directions 

protected_regression = linear_model.LinearRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train)
sensetive_directions = protected_regression.coef_

sensetive_directions = scipy.linalg.orth(sensetive_directions.T).T
for i, s in enumerate(sensetive_directions):
    while np.linalg.norm(s) != 1:
        s = s/ np.linalg.norm(s)
    sensetive_directions[i] = s





unprotected_directions = utils.projection_matrix(sensetive_directions)
protected_directions = utils.projection_matrix2(sensetive_directions)



# Casing to tensor 
y_train, y_test = y_train.astype('int32'), y_test.astype('int32')
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)
unprotected_directions = tf.cast(unprotected_directions, dtype = tf.float32)
protected_directions = tf.cast(protected_directions, dtype = tf.float32)
sensetive_directions = tf.cast(sensetive_directions, dtype = tf.float32)



graph = model.model # use for unfair algo
#graph = cl.Classifier(init_graph, tf.matmul(x_unprotected_train, unprotected_directions), 
#                        y_train, tf.matmul(x_unprotected_test, unprotected_directions), y_test, num_steps = 10000) # for fair algo



def sample_perturbation(data_point, regularizer = 5, learning_rate = 5e-2, num_steps = 200):
    x, y = data_point
    x = tf.reshape(x, (1, -1))
    y = tf.reshape(y, (1, -1))
    x_start = x
    x_fair = x
    x_base = x
    #x += tf.cast(np.random.normal(size=(1, 39)), dtype = tf.float32)*1e-9
    for _ in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x_fair)
            prob = graph(x_fair)
            perturb = utils.unprotected_direction(x_fair-x_start, sensetive_directions)
            loss = utils.EntropyLoss(y, prob)  - regularizer * tf.norm(perturb)**2

        gradient = g.gradient(loss, x_fair)
        x_fair = x_fair + learning_rate * gradient#utils.protected_direction(gradient, sensetive_directions)

    for _ in range(num_steps):
        with tf.GradientTape() as g:
            g.watch(x_base)
            prob = graph(x_base)
            perturb = x_base-x_start
            loss = utils.EntropyLoss(y, prob)  - regularizer * tf.norm(perturb)**2

        gradient = g.gradient(loss, x_base)
        x_base = x_base + learning_rate * gradient

    

    return_loss = utils.EntropyLoss(y, graph(x_fair)) / utils.EntropyLoss(y, graph(x_base))
    
    return return_loss.numpy()




cpus = mp.cpu_count()
print(f'Number of cpus : {cpus}')
start_time = time.time()
with mp.Pool(cpus) as pool:
    perturbed_test_samples = pool.map(sample_perturbation, zip(x_unprotected_test[:100], y_test[:100]))

Number of cpus : 8


In [2]:
perturbed_test_samples

[3.9691126,
 13.925428,
 2.2100072,
 11.714804,
 41.950523,
 4.8249426,
 1.5018934,
 17.350945,
 46.654392,
 24.01961,
 21.631062,
 1.4258924,
 28.703575,
 49.158707,
 23.06944,
 13.418418,
 21.703249,
 4.696373,
 34.581955,
 38.23759,
 21.755995,
 29.909342,
 3.3940756,
 1.4462055,
 14.908122,
 3.8899705,
 12.659021,
 10.2832365,
 46.990986,
 10.470563,
 19.086178,
 1.4684105,
 4.788865,
 1.6900117,
 13.212728,
 19.994469,
 12.793514,
 16.327148,
 14.604453,
 2.1824641,
 7.4576597,
 1.4507582,
 1.5765291,
 8.616739,
 12.0623665,
 8.860075,
 45.78071,
 11.457769,
 3.4296248,
 7.197546,
 49.55938,
 18.635733,
 1.4855579,
 4.1576357,
 1.6539228,
 4.264617,
 24.922611,
 4.329921,
 7.640199,
 5.394464,
 5.642277,
 3.3313859,
 6.9149437,
 2.206439,
 5.265876,
 17.215036,
 5.276814,
 3.7212853,
 12.701902,
 9.59495,
 43.202404,
 1.2672855,
 23.82768,
 2.075856,
 7.544819,
 5.687431,
 17.969654,
 2.9267294,
 4.2355456,
 2.102731,
 5.9052534,
 44.543736,
 4.636503,
 2.069585,
 45.07301,
 22.53

In [3]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import model
import utils
import time
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import scipy
plt.ioff()


seed = 1
tf.random.set_seed(seed)
np.random.seed(seed)
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))







In [9]:



## Running linear regression to get sensetive directions 

protected_regression = linear_model.LogisticRegression(fit_intercept = False)
protected_regression.fit(x_unprotected_train, x_protected_train[:, 0])
sensetive_directions = []
sensetive_directions.append(protected_regression.coef_[0])

protected_regression.fit(x_unprotected_train, x_protected_train[:, 1])
sensetive_directions.append(protected_regression.coef_[0])

In [11]:
sensetive_directions=np.array(sensetive_directions)

array([[-8.72337824e-02,  4.43155554e-02,  7.94081980e-02,
         7.41708183e-02,  2.79834553e-01,  2.89992539e-01,
        -2.82469875e-01, -5.27195222e-02,  8.62391393e-01,
         6.11585088e-01,  2.81603683e-02, -4.83852113e-01,
         1.94264721e-01, -2.66407228e-01,  2.94272156e-01,
         4.90837119e-01,  6.78182725e-01,  2.64240948e-01,
        -6.82302564e-01, -1.54839649e+00,  9.28693331e-01,
         1.84398566e+00, -6.60123060e-01,  1.76797052e+00,
         1.38518756e+00,  9.80251082e-02, -6.78558724e-01,
        -2.52616422e+00, -8.05093252e-01,  6.31757178e-01,
        -7.08884376e-01, -6.61131552e-01,  1.90582019e+00,
         7.43438550e+00,  9.46283637e-02,  9.76834406e-02,
         1.27018253e-01, -1.18397476e+00, -5.59665292e+00],
       [-8.56607133e-02,  5.03699733e-02, -4.87509960e-04,
         2.76424404e-02, -1.81455095e-03, -3.09012913e-01,
         1.88892616e-01,  5.58788899e-01,  9.18391462e-01,
         1.00298973e+00,  1.93522269e-01,  5.21701744e-